# Running a 5-Point Calibration Scan Without OSO Using SPFRx

**Currently, this notebook can only run on Talon1, due to that board being the one connected to the SPFRx ,RXPU must also be checked out**

This notebook adapts the [Five Point Calibration Scan Controls](https://gitlab.com/ska-telescope/ska-jupyter-scripting/-/blob/main/notebooks/observing/MID_five_point_calibration_scan_controls.ipynb?ref_type=heads) notebook to work with PSI. Running this notebook requires a PSI namespace to be spun up with the CI argument `TMC_ENABLED` set to `true`. As this notebook also utilizes SPFRx, the namespace should also be set up with `DISH_LMC_ENABLED` set to true as well.

## 1 Setup

### 1.1 Environment Setup

Start by importing all the libraries needed for this notebook

In [ ]:
import json
import os
import random
import time
from datetime import date
from time import sleep

from astropy.time import Time
from tango import DevFailed, DeviceProxy

Until notebook clean up is merged need to use this for waiting for state

In [ ]:
spinner = ["⣾", "⣽", "⣻", "⢿", "⡿", "⣟", "⣯", "⣷"]


def wait_for_state(device: DeviceProxy, desired_state, break_on_error=True) -> None:
    """Poll a tango device until either the given observation state is reached, or it throws an error.
    Arguments:
    device -- Tango Device to check
    desired_state -- The state which to break upon getting (number or state)
    break_on_error -- If set to False, will keeping running when getting an error status.
    """
    spinL = 0
    poll = 1
    while device.obsState != desired_state:
        if spinL < len(spinner) - 1:
            spinL += 1
        else:
            spinL = 0
        sleep(0.5)
        print(
            "\r",
            f"{spinner[spinL]} Poll# {poll}: Current state is {device.obsState.name}, waiting for {desired_state}...",
            end="",
        )
        if device.obsState == 9 and break_on_error:
            break
        poll += 1
    print(f"\nFinished with: {device.obsState.name}")

### 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [ ]:
!kubectl get ns | grep ska-mid-psi

And load it into the variables for the notebook:

In [ ]:
namespace = "ci-ska-mid-psi-1370357924-alexschell"  # set to desired NS
simulation_mode = 0  # set to 1 to run in sim mode
server = "ska-sdp-kafka." + namespace + ".svc.cluster.local:9092"

In [ ]:
print("Now the displays for debugging can be used:")
print(f"https://142.73.34.170/{namespace}/signal/display/")
print(f"https://142.73.34.170/{namespace}/taranta/devices")
print(f"https://142.73.34.170/{namespace}/taranta/dashboard?id=666cb28b5e5d4f0012197e5f&mode=run")

Next, load all the other vars the notebook will use. These should not need to be changed for this run.

In [ ]:
# Fully Qualified Domain Names for the devices to set up proxies
CSP_CONTROLLER_FQDN = "mid-csp/control/0"
CSP_SUBARRAY_FQDN = "mid-csp/subarray/01"

TARGET_BOARDS_LIST = [1]

CENTRAL_NODE_FQDN = "ska_mid/tm_central/central_node"
LEAF_NODE_FQDN = "ska_mid/tm_leaf_node/csp_master"
LEAF_NODE_SUBARRAY_FQDN = "ska_mid/tm_leaf_node/csp_subarray01"
TMC_SUBARRAY_FQDN = "ska_mid/tm_subarray_node/1"

CBF_CONTROLLER_FQDN = "mid_csp_cbf/sub_elt/controller"
CBF_SUBARRAY_FQDN = "mid_csp_cbf/sub_elt/subarray_01"

DISH_MANAGER_FQDN = "mid-dish/dish-manager/ska001"

DEPLOYER_FQDN = "mid_csp_cbf/ec/deployer"

SPFRX_FQDN = "ska001/spfrxpu/controller"

# Tango host environment variable
TANGO_HOST = "databaseds-tango-base." + namespace + ".svc.cluster.local:10000"

# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIG = os.path.join(DATA_DIR, "mid_telescope/cbf")
CSP_CONFIG = os.path.join(DATA_DIR, "mid_telescope/csp")
TMC_CONFIG = os.path.join(DATA_DIR, "mid_telescope/tmc")
HW_CONFIG = os.path.join(COMMON_CONFIG, "hw_config")
SLIM_CONFIG = os.path.join(COMMON_CONFIG, "slim_config")
CBF_INPUT_DIR = os.path.join(COMMON_CONFIG, "cbf_input_data")
# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]

Next, set the environment arg for TANGO HOST:

In [ ]:
print("Will be using HOST: ", TANGO_HOST)
os.environ["TANGO_HOST"] = TANGO_HOST

With all the file paths defined the JSON files can be loaded in and checked.

In [ ]:
print("Getting files...")

INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG, "sys_params/initial_system_param.json")
ASSIGN_RESOURCES_FILE = os.path.join(TMC_CONFIG, "assign_resources.json")
ASSIGN_CSP_RESOURCES_FILE = os.path.join(CSP_CONFIG, "assign_resources.json")
CONFIGURE_SCAN_FILE = os.path.join(TMC_CONFIG, "configure_scan.json")
CSP_DELAY_MODEL_FILE = os.path.join(TMC_CONFIG, "delay_model.json")
DISH_CONFIG_FILE = f"{COMMON_CONFIG}/sys_params/load_dish_config.json"

START_CHANNEL = 0
END_CHANNEL = 14860
START_PORT = 21000

SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    DISH_CONFIG_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

print("Done")

Next, set up the hw config to match the boards in use:

In [ ]:
config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

receptor_ids = ["SKA001"]

If the HW config file exists, it is loaded into the pod:

In [ ]:
!kubectl cp $HW_CONFIG_FILE $namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

### 1.3 Create Device Proxies

Using the FQDNs set earlier, and with the pod spun up, create device proxies to the devices used and check the connection to them.

In [ ]:
# CSP Devices
csp_controller = DeviceProxy(CSP_CONTROLLER_FQDN)
print(f"CSP Controller: {csp_controller.Status()}")
csp_subarray = DeviceProxy(CSP_SUBARRAY_FQDN)
print(f"CSP Subarray: {csp_subarray.Status()}")

# TMC Devices
tmc_central_node = DeviceProxy(CENTRAL_NODE_FQDN)
print(f"Central Node: {tmc_central_node.Status()}")
leaf_node_subarray = DeviceProxy(LEAF_NODE_SUBARRAY_FQDN)
leaf_node_master = DeviceProxy(LEAF_NODE_FQDN)
print(f"Leaf Subarray Node: {leaf_node_subarray.Status()}")
tmc_subarray = DeviceProxy(TMC_SUBARRAY_FQDN)
print(f"TMC subarray Node: {tmc_subarray.Status()}")

# Deployer for setup and BITE for data mocking
deployer = DeviceProxy(DEPLOYER_FQDN)

dish_manager = DeviceProxy(DISH_MANAGER_FQDN)

# CBF Device
cbf_subarray = DeviceProxy(CBF_SUBARRAY_FQDN)
print(f"CBF subarray: {cbf_subarray.Status()}")

### 1.4 Downloading Requirements via the Deployer

First, set the board to deploy to and turn on the deployer device.

In [ ]:
deployer.On()
deployer.targetTalons = TARGET_BOARDS_LIST
print(deployer.targetTalons)
deployer.generate_config_jsons()

Once started and configured, the required devices can then be downloaded.

In [ ]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(3000)

Now configure the device database with the downloaded devices:

In [ ]:
deployer.configure_db()

## 2 Setting up the Devices

### 2.1 Set up TMC Central Node

Now setup the CSP proxy, and load in the relevant values:

In [ ]:
tmc_central_node.adminMode = 0
tmc_central_node.simulationMode = simulation_mode

In [ ]:
# Set relevant values on the mid-csp controller
csp_controller.adminMode = 0
csp_subarray.adminMode = 0
sleep(1)
csp_controller.cbfSimulationMode = simulation_mode
csp_subarray.SimulationMode = simulation_mode
sleep(1)
if (
    csp_controller.read_attribute("adminMode").value == 0
    and csp_controller.read_attribute("cbfSimulationMode").value == 0
):
    print("Set to simulation mode off.")
elif (
    csp_controller.read_attribute("adminMode").value == 0
    and csp_controller.read_attribute("cbfSimulationMode").value == 1
):
    print("Set to simulation mode on.")
else:
    print("Error, couldn't set values!")

Next load in the dish config file to the central node:

In [23]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)
dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))
sleep(5)
print(f"Config sent:\n{dish_config_json}")
print(f"Config set:\n{leaf_node_master.sourceDishVccConfig}")
print(f"{leaf_node_master.dishVccConfig}")

Config sent:
{'interface': 'https://schema.skao.int/ska-mid-cbf-initsysparam/1.0', 'tm_data_sources': ['car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata'], 'tm_data_filepath': 'instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json'}
Config set:
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"], "tm_data_filepath": "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"}
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 1}, "SKA036": {"vcc": 2, "k": 1}, "SKA063": {"vcc": 3, "k": 1}, "SKA100": {"vcc": 4, "k": 1}}}


In [ ]:
print(tmc_central_node.isDishVccConfigSet)

### 2.3 Turning on the Telescope via TMC

Now, turn the telescope itself on (TelescopeOn may complain about VCC config, have to wait for it to resolve):

In [ ]:
tmc_central_node.set_timeout_millis(100000)
tmc_central_node.TelescopeOn()
sleep(100)
startup_time = 0
while int(tmc_central_node.telescopeState) != 0:
    print(f"Telescope is starting up, {startup_time} seconds elapsed.")
    sleep(5)
    startup_time += 5
    print("\r", end="")
print(f"telescope has started after {startup_time} seconds.")
print(tmc_central_node.status())
print(csp_controller.status())

In [ ]:
print("Verify device states:")
print(f"\tTMC central node: {tmc_central_node.State()}. Should be ON")
print(f"\tTMC subarray node: {tmc_subarray.State()}. Should be ON")
print(f"\tCSP controller node: {csp_controller.State()}. Should be ON")
print(f"\tCSP subarray node: {csp_subarray.State()}. Should be ON")
print(f"\tCBF subarray node: {cbf_subarray.State()}. Should be ON")

## 3 Controlling the 5-Point Scan from TMC

### 3.1 Set up JSON Files

Start by getting the base file to use to assign resources to TMC

In [ ]:
with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as assign_file:
    basic_assign_json = json.load(assign_file)

Load in the config file for TMC

In [ ]:
with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as config_file:
    basic_configure_json = json.load(config_file)

Now overwrite the vars that need to be modified.

In [ ]:
today = date.today().strftime("%Y%m%d")
random_id = random.randint(0, 99999)
beams = [
    {"beam_id": "vis0", "function": "visibilities"},
    {"beam_id": "pss1", "search_beam_id": 1, "function": "pulsar search"},
    {"beam_id": "pss2", "search_beam_id": 2, "function": "pulsar search"},
    {"beam_id": "pst1", "timing_beam_id": 1, "function": "pulsar timing"},
    {"beam_id": "pst2", "timing_beam_id": 2, "function": "pulsar timing"},
    {"beam_id": "vlbi1", "vlbi_beam_id": 1, "function": "vlbi"},
]

port_list = []
run = 0
while run * 20 + START_CHANNEL <= END_CHANNEL:
    port_list.append([run * 20 + START_CHANNEL, START_PORT + run])
    run += 1

EB_ID = f"eb-testtmc-{today}-{random_id:05d}"
PB_ID = f"pb-testrealtime-{today}-{random_id:05d}"

# Configure the assign resources JSON for TMC
basic_assign_json["sdp"]["processing_blocks"][0]["parameters"]["queue_connector_configuration"][
    "exchanges"
][0]["source"]["servers"] = server
basic_assign_json["dish"]["receptor_ids"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], TARGET_BOARDS_LIST)
)
basic_assign_json["sdp"]["execution_block"]["beams"] = beams
basic_assign_json["sdp"]["processing_blocks"][0]["pb_id"] = PB_ID
basic_assign_json["sdp"]["execution_block"]["eb_id"] = EB_ID
basic_assign_json["sdp"]["resources"]["receptors"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], TARGET_BOARDS_LIST)
)

# Configure the configuration JSON for TMC
basic_configure_json["csp"]["common"]["config_id"] = "1 receptor, band 1, 1 FSP, no options"
basic_configure_json["csp"]["subarray"]["subarray_name"] = "1 receptor"

# Set the FSP ID to match the board and set networking
basic_configure_json["csp"]["cbf"]["fsp"][0]["fsp_id"] = TARGET_BOARDS_LIST[0]
basic_configure_json["csp"]["cbf"]["fsp"][0]["frequency_slice_id"] = TARGET_BOARDS_LIST[0]

print("Modified assign json:")
print(json.dumps(basic_assign_json, indent=1))
print("===================")
print(json.dumps(basic_configure_json, indent=1))
print("===================")

### 3.2 Assign Resources to the Telescope and Configure Subarray

Now, use TMC to pass the configurations down to the devices used (SDP,CSP...)

In [ ]:
tmc_subarray.AssignResources(json.dumps(basic_assign_json))
wait_for_state(tmc_subarray, 2)

After this step, check the -sdp namespace to ensure the vis pod is spun up:

In [ ]:
# Grab the pod name
!kubectl -n $namespace-sdp get pods | grep vis-receive

Now configure the tracking table. (explain AJ wizardry here?)

In [ ]:
current_pointing = dish_manager.achievedPointing
current_az = current_pointing[1]
current_el = current_pointing[2]

astropy_time = Time((time.time() + 120), format="unix")

current_time_tai_s = astropy_time.unix_tai

# Directions to move values
az_dir = 1 if current_az < 350 else -1
el_dir = 1 if current_el < 80 else -1

track_table = [
    current_time_tai_s + 3,
    current_az + 1 * az_dir,
    current_el + 1 * el_dir,
    current_time_tai_s + 5,
    current_az + 2 * az_dir,
    current_el + 2 * el_dir,
    current_time_tai_s + 7,
    current_az + 3 * az_dir,
    current_el + 3 * el_dir,
    current_time_tai_s + 9,
    current_az + 4 * az_dir,
    current_el + 4 * el_dir,
    current_time_tai_s + 11,
    current_az + 5 * az_dir,
    current_el + 5 * el_dir,
]

dish_manager.programTrackTable = track_table

## 4 Running the 5-Point Scan

Now the scans can actually be run:

### 4.1 Configure the TMC

Now, send the configuration to TMC, and wait for it to go READY:

In [ ]:
tmc_subarray.Configure(json.dumps(basic_configure_json))
wait_for_state(tmc_subarray, 4)

### 4.2 Run Scan 1: No Offset

First, set up the vis pod to monitor for packets, running this output on the dev server.

In [ ]:
!echo kubectl exec -n $namespace-sdp -ti $vis_pod -- bash

Once in, install `tcpdump` and run it by copying this command to the bash:

Then have the subarray scan without setting an offset, such that it scans at [0,0].

In [ ]:
long_basic_scan_json = {
    "interface": "https://schema.skao.int/ska-tmc-scan/2.1",
    "transaction_id": "txn-....-00003",
    "scan_id": 1,
}

In [ ]:
tmc_subarray.Scan(json.dumps(long_basic_scan_json))
wait_for_state(cbf_subarray, 5)
print("Central Scan has started, giving it some time to run...")
sleep(100)
tmc_subarray.EndScan()
wait_for_state(cbf_subarray, 4)

Watch the logs to ensure the proper packet lengths, Once this passes, the offset scans can be run.

### 4.3 Run Offset Scans 2-5

**This is currently blocked as reconfiguration causes low-level TANGO memory allocation issues**

Now, loop through the offset scans, using the combos to set the scan json. For each, configure it by passing in JSON, then scan again, letting it run before stopping.

In [ ]:
count = 4
for offset in SCAN_COMBOS:
    print(f"setting offset to: {offset}")

    # Configure for this offset scan
    partial_configure_json = {
        "interface": "https://schema.skao.int/ska-tmc-configure/2.2",
        "transaction_id": f"txn-....-0000{count}",
        "scan_id": count,
        "pointing": {"target": {"ca_offset_arcsec": offset[0], "ie_offset_arcsec": offset[1]}},
        "tmc": {"partial_configuration": True},
    }
    print("Partial Config to load:")
    print(json.dumps(partial_configure_json, indent=1))
    print(".......")
    leaf_node_subarray.Configure(json.dumps(partial_configure_json))
    wait_for_state(cbf_subarray, 4)
    sleep(10)
    # Send the Scan command along with relevant JSON,incrementing the scan ID and transaction ID
    partial_scan_json = {
        "interface": "https://schema.skao.int/ska-tmc-scan/2.1",
        "transaction_id": f"txn-....-0000{count+4}",
        "scan_id": 2 + count,
    }
    leaf_node_subarray.Scan(json.dumps(partial_scan_json))
    wait_for_state(cbf_subarray, 5)
    # let the scan run for a bit...
    sleep(200)
    leaf_node_subarray.EndScan()
    wait_for_state(cbf_subarray, 4)

    count += 1
    print("============================")
print("Done offsets!")

## 5 Cleanup:

In [ ]:
tmc_subarray.GoToIdle()

In [ ]:
tmc_subarray.ReleaseAllResources()

In [ ]:
tmc_subarray.ObsReset()